In [1]:
import numpy as np
import open3d as o3d
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from tabulate import tabulate

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


切片

In [11]:
def slice_pts(pts_file, slice_distance):
    # Read the PTS file
    points = np.loadtxt(pts_file)

    # Split points into coordinates and colors
    coordinates = points[:, :3]

    # Get the minimum and maximum Z coordinates of the point cloud
    min_z = np.min(coordinates[:, 2])
    max_z = np.max(coordinates[:, 2])

    # Calculate the number of slices
    num_slices = int((max_z - min_z) / slice_distance)

    # Create a list to store sliced point clouds
    sliced_point_clouds = []

    # Slice the point cloud based on the slice distance
    for i in range(num_slices+1):
        # Calculate the height range of the slice
        z_min = min_z + i * slice_distance
        z_max = z_min + slice_distance

        # Extract points within the slice range
        sliced_coordinates = coordinates[(coordinates[:, 2] >= z_min) & (coordinates[:, 2] < z_max)]

        # Create a sliced point cloud
        sliced_point_cloud = o3d.geometry.PointCloud()
        sliced_point_cloud.points = o3d.utility.Vector3dVector(sliced_coordinates)

        # Add the sliced point cloud to the list
        sliced_point_clouds.append(sliced_point_cloud)

    return sliced_point_clouds


In [12]:
# 讀pts檔
pts_file = 'C:/Users/Justine Huang/Python/147_0000.pts'
# 切片距離
slice_distance = 10
# 切片
sliced_point_clouds = slice_pts(pts_file, slice_distance)
# print(len(sliced_point_clouds))
# 分別存取切片
for i, sliced_point_cloud in enumerate(sliced_point_clouds):
    output_file = f'slice20_{i}.pts'
    np.savetxt(output_file, np.asarray(sliced_point_cloud.points))

算點數

In [13]:
num_points = 0
for i, sliced_point_cloud in enumerate(sliced_point_clouds):
    points = np.loadtxt(f'C:/Users/Justine Huang/Python/slice20_{i}.pts')
    # 計算點數
    num_points += len(points)
print("點數:", num_points)

點數: 8000


算密度

In [ ]:
min_coords = np.min(points[:, :3], axis=0)  # 找xyz最小值
max_coords = np.max(points[:, :3], axis=0)  # 找xyz最大值
bounding_box_size = max_coords - min_coords
volume = np.prod(bounding_box_size) # 算出體積

density = num_points / volume

print("密度:", density)

點順序

In [ ]:
# 以Z軸最小的點為起點
min_z_index = np.argmin(points[:, 2])
start_index = min_z_index
start_point = points[start_index, :3]
print("起點:",start_point)

# 計算每點相對於起點的角度
def angle(point):
    angle_rad = 2 * np.pi - np.arctan2(point[1] - start_point[1], point[0] - start_point[0])
    angle_deg = np.degrees(angle_rad)
    angle_adjusted = angle_deg % 360.0
    return angle_adjusted

angles = np.array([angle(point) for point in points[:, :3]])

# 排序點和角度
sorted_indices = np.argsort(angles)
sorted_points = points[sorted_indices]
sorted_angles = angles[sorted_indices]

# 計算距離
distances = np.linalg.norm(sorted_points[1:, :3] - sorted_points[:-1, :3], axis=1)
distances = np.insert(distances, 0, 0) 
average_distance = np.mean(distances)
max_distance = np.max(distances)
print(f"平均距離:{average_distance}, 最遠距離:{max_distance}")
dis_dif = distances - average_distance

data = []
for i in range(num_points):
    point = sorted_points[i, :3]
    angle = sorted_angles[i]
    distance = distances[i]
    dif = dis_dif[i]
    if i == 0:
        distance = 0
        dif = 0
    data.append([point, angle, distance, dif])

columns = ["座標","角度","距離","與平均距離差距"]
df = pd.DataFrame(data, columns=columns)

# Print the aligned DataFrame using tabulate
table = tabulate(df, headers='keys', tablefmt='plain', stralign='left', showindex=True)
print(table)